In [ ]:
from sklearn import cluster
from sklearn.cluster import KMeans

from functions import *
from KNN import *
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

owd = os.getcwd()
os.chdir('brainnetworks/CSVdata/')

selection = 'sex' #default creativity

data = []
order = []
for file in os.listdir():
    data.append(normalize(pd.read_csv(file, header=None).values).flatten())
    order.append(file[:9])

# GET LABELS
sex = getLabel(0, order)
math = getLabel(1, order)
creativity = getLabel(2, order)

nanMask = ~(sex.isna() | math.isna() | creativity.isna())
sex = sex[nanMask]
math = math[nanMask]
creativity = creativity[nanMask]
data = [data[i] for i in range(0,len(data)) if nanMask[i]]

# Used to assist in score calculation function to prevent wrong function/errors
label_set = sex if selection == 'sex' else math if selection == 'math' else creativity
n_component_list = [*range(10,101,10)]
for n_components in n_component_list:
    all_scores = []
    for seed in range(0, 10):
        X_train, X_test, y_train, y_test = train_test_split(data, label_set, test_size=0.2, random_state=seed)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        #####################################################################
        # PCA

        # Manual PCA
        X_train_mean = X_train - np.mean(X_train)
        covar = np.cov(X_train_mean, rowvar=False)
        eigVal, eigVec = np.linalg.eig(covar) # maybe use eigh instead rather than eig, because a covariance matrix is supposed to be symmetric
        q = [(eigVal[i], eigVec[:,i]) for i in range(0, len(eigVal))]
        q.sort(key = lambda x: x[0], reverse=True)
        matrix = q[0][1]
        for i in range (1, n_components):
            matrix = np.append(matrix, q[i][1])
        matrix = matrix.reshape(n_components,-1)
        X_train_pca = np.matmul(X_train, matrix.T)
        X_test_pca = np.matmul(X_test, matrix.T)

        explained_variance_ratio = [float(eigVal[i] / sum(eigVal)) for i in range(0, n_components)]
        #for i in range(n_components):
        #    print('Percentage of variance explained by PC {}: {}'.format(i+1, explained_variance_ratio[i]))
        #print('Total variance explained by 20 PCs: {}'.format(np.sum(explained_variance_ratio)))

        pca_fig, pca_ax = plt.subplots(1)
        pca_ax.plot(range(1, n_components+1), [sum(explained_variance_ratio[0:x+1]) for x in range(0, len(explained_variance_ratio))])
        pca_ax.set_xlabel('Num principle components')
        pca_ax.set_ylabel('Fraction of Total Variance Explained')

        # Manual KNN
        distance_types = [1,2]
        k_values = range(9,36,2)

        for dstType in distance_types:
            for k in k_values:
                pred_labels = []
                for i in range(0,len(y_test)):
                    pred_labels.append(knn(X_test_pca[i], y_train, X_train_pca, k, dstType))
                    #print('Document ' + str(i) + ' groundtruth ' + str(y_test[i]) + ' predicted as ' + str(pred_labels[-1]))

                # Accuracy is calculated as the average of 1 - (|ytrue-ypred| / ytrue) for all test points
                # used for creativity and math
                scores = []
                if selection == 'sex':
                    # due to the nature of binary selection, the round still guarantees pred_labels[i] 
                    # is the most common of the nearest k labels
                    scores = [(1 if y_test[i] == round(pred_labels[i]) else 0) for i in range(0, len(y_test))]
                else:
                    scores = [(1 - (abs(y_test[i] - pred_labels[i]) / y_test[i])) for i in range(0, len(y_test))]
                accuracy_score = sum(scores) / len(scores)
                print("Seed:", seed, "Distance type:", dstType, "K value:", k, ", Accuracy Score:", accuracy_score)
                all_scores.append((seed, dstType, k, accuracy_score))

    scores_df = pd.DataFrame(all_scores, columns=['Seed', 'Distance Type', 'K', 'Accuracy'])
    scores_df.to_csv('../../outdata/'+selection+'/'+str(n_components)+'.csv')
    scores_df.sort_values(['Distance Type', 'K', 'Accuracy'], ascending=False).groupby(['Distance Type', 'K']).mean()
os.chdir(owd)
    

In [19]:
import pandas as pd
import numpy as np

pca_components = [*range(10,101,10)]
pcas = []
for pca in pca_components:
    pcas.append(pd.read_csv('outdata/sex/'+str(pca) + '.csv', index_col=0))

data_df = pcas[0][['Seed', 'Distance Type', 'K']]
for i, n_components in enumerate(pca_components):
    data_df[n_components] = pcas[i]['Accuracy']

data_df

,Seed,Distance Type,K,10,20,30,40,50,60,70,80,90,100
0,0,1,9,0.478261,0.608696,0.608696,0.782609,0.608696,0.608696,0.652174,0.652174,0.695652,0.695652
1,0,1,11,0.608696,0.652174,0.565217,0.695652,0.608696,0.608696,0.652174,0.565217,0.565217,0.565217
2,0,1,13,0.739130,0.565217,0.565217,0.695652,0.695652,0.652174,0.739130,0.652174,0.695652,0.695652
3,0,1,15,0.652174,0.521739,0.608696,0.695652,0.652174,0.652174,0.782609,0.695652,0.652174,0.652174
4,0,1,17,0.608696,0.565217,0.565217,0.652174,0.695652,0.652174,0.782609,0.652174,0.652174,0.652174
5,0,1,19,0.608696,0.565217,0.565217,0.565217,0.695652,0.695652,0.782609,0.695652,0.652174,0.652174
6,0,1,21,0.652174,0.608696,0.521739,0.608696,0.565217,0.739130,0.782609,0.739130,0.695652,0.695652
7,0,1,23,0.652174,0.565217,0.565217,0.565217,0.608696,0.782609,0.782609,0.695652,0.782609,0.782609
8,0,1,25,0.695652,0.652174,0.608696,0.478261,0.565217,0.782609,0.739130,0.695652,0.652174,0.652174
9,0,1,27,0.652174,0.652174,0.565217,0.521739,0.608696,0.739130,0.739130,0.695652,0.521739,0.521739


In [38]:
data_df.groupby(['Distance Type', 'K']).mean()[20].to_frame()

20
Distance Type K           
1             9   0.582609
              11  0.543478
              13  0.526087
              15  0.517391
              17  0.534783
              19  0.543478
              21  0.534783
              23  0.521739
              25  0.552174
              27  0.539130
              29  0.547826
              31  0.560870
              33  0.569565
              35  0.552174
2             9   0.556522
              11  0.543478
              13  0.539130
              15  0.539130
              17  0.569565
              19  0.595652
              21  0.578261
              23  0.595652
              25  0.582609
              27  0.543478
              29  0.547826
              31  0.539130
              33  0.560870
              35  0.560870

In [73]:
#Graph for Sex Prediction
data = data_df.groupby(['Distance Type', 'K']).mean()[20].values
df = pd.DataFrame([data, [*range(9,36,2)]*2, ['SSD']*14+['ABV']*14], index=['Accuracy', 'K Neighbors', 'Distance Metric']).T
import plotly.express as px
fig = px.line(df, x='K Neighbors', y='Accuracy', color='Distance Metric')
fig.show()